In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

#### 2019 + 2020

In [3]:
data19 = pd.read_csv("CT_Crash_Data_2019.csv")
data20 = pd.read_csv("CT_Crash_Data_2020.csv")

In [4]:
data19.shape

(112011, 108)

In [6]:
data = data19.append(data20, sort = False)

In [7]:
data.shape

(141936, 108)

In [8]:
data['Crash_Fatal_Fl'] = np.where((data['Crash Severity Text Format'] == 'Fatal (Kill)'),1,0)
dataSub = data[['CrashId', 'Crash_Fatal_Fl','Date Of Crash']]
# may fix later
dataSub['Death_Cnt'] = 0
dataSub['Date Of Crash'] = pd.to_datetime(dataSub['Date Of Crash'])

# keep only before April
dataSub = dataSub[dataSub['Date Of Crash'] < '2020-06-01']

In [9]:
dataSubAgg = dataSub.copy()

In [19]:
dataSubAgg['yearMonth'] = dataSubAgg['Date Of Crash'].dt.strftime('%Y-%m')
dataSubAgg['Year'] = dataSubAgg['Date Of Crash'].dt.strftime('%Y')
dataSubAgg['Month'] = dataSubAgg['Date Of Crash'].dt.strftime('%m')
dataSubAgg = dataSubAgg.drop(['Date Of Crash'], axis = 1)

In [20]:
dataSubAgg.rename(columns={"CrashId": "Crash_ID"}, inplace = True)

In [21]:
dataSubAgg.head()

,Crash_ID,Crash_Fatal_Fl,Death_Cnt,yearMonth,Year,Month
0,578496,0,0,2019-01,2019,01
1,578767,0,0,2019-01,2019,01
2,578768,0,0,2019-01,2019,01
3,578769,0,0,2019-01,2019,01
4,579168,0,0,2019-01,2019,01


## YearMonth

In [22]:
dataSubYearMonth = dataSubAgg.groupby(['Year','Month']).agg({'Crash_ID': 'count',
                                                                   'Crash_Fatal_Fl': 'sum',
                                                                   'Death_Cnt': 'sum',
                                                                   'yearMonth': 'max'}).reset_index()

In [23]:
dataSubYearMonth['crashPctChange'] = dataSubYearMonth.Crash_ID.pct_change()
dataSubYearMonth['deathCntPctChange'] = dataSubYearMonth.Death_Cnt.pct_change()
dataSubYearMonth['fatalPctChange'] = dataSubYearMonth.Crash_Fatal_Fl.pct_change()
dataSubYearMonth['State']='Connecticut'
dataSubYearMonth['Country']='United States'

In [24]:
dataSubYearMonth['deathRate']=dataSubYearMonth['Death_Cnt']/dataSubYearMonth['Crash_ID']
dataSubYearMonth['fatRate']=dataSubYearMonth['Crash_Fatal_Fl']/dataSubYearMonth['Crash_ID']

In [25]:
dataSubYearMonth['deathRatePctChange'] = dataSubYearMonth.deathRate.pct_change()
dataSubYearMonth['fatRatePctChange'] = dataSubYearMonth.fatRate.pct_change()

In [38]:
with open("CT_dataSubYearMonth.csv",'w',newline='') as outputfile:
    dataSubYearMonth.to_csv(outputfile, header=True, index = False) 

In [27]:
datacheck = pd.read_csv('CT_dataSubYearMonth.csv')

In [28]:
datacheck.shape

(17, 15)

## MonthYear

In [29]:
dataSubMonthYear = dataSubAgg.groupby(['Month','Year']).agg({'Crash_ID': 'count',
                                                                   'Crash_Fatal_Fl': 'sum',
                                                                   'Death_Cnt': 'sum',
                                                                   'yearMonth': 'max'}).reset_index()

In [30]:
dataSubMonthYear['crashPctChange'] = dataSubMonthYear.groupby('Month')['Crash_ID'].apply(lambda x: x.div(x.iloc[0]).subtract(1))
dataSubMonthYear['deathCntPctChange'] = dataSubMonthYear.groupby('Month')['Death_Cnt'].apply(lambda x: x.div(x.iloc[0]).subtract(1))
dataSubMonthYear['fatalPctChange'] = dataSubMonthYear.groupby('Month')['Crash_Fatal_Fl'].apply(lambda x: x.div(x.iloc[0]).subtract(1))
dataSubMonthYear['State']='Connecticut'
dataSubMonthYear['Country']='United States'

In [31]:
dataSubMonthYear['deathRate']=dataSubMonthYear['Death_Cnt']/dataSubMonthYear['Crash_ID']
dataSubMonthYear['fatRate']=dataSubMonthYear['Crash_Fatal_Fl']/dataSubMonthYear['Crash_ID']

In [32]:
dataSubMonthYear['deathRatePctChange'] = dataSubMonthYear.groupby('Month')['deathRate'].apply(lambda x: x.div(x.iloc[0]).subtract(1))
dataSubMonthYear['fatRatePctChange'] = dataSubMonthYear.groupby('Month')['fatRate'].apply(lambda x: x.div(x.iloc[0]).subtract(1))

In [39]:
with open("CT_dataSubMonthYear.csv",'w',newline='') as outputfile:
    dataSubMonthYear.to_csv(outputfile, header=True, index = False) 

In [34]:
datacheck = pd.read_csv('CT_dataSubMonthYear.csv')
datacheck.shape

(17, 15)

#### The december tied data
In this section we compare the all the months we have in 2020 to the most recent "normal month" which is December.

In [35]:
compare = dataSubYearMonth[['yearMonth', 'Crash_ID', 'Crash_Fatal_Fl']][(dataSubYearMonth.yearMonth >= '2019-12') & (dataSubYearMonth.yearMonth < '2020-06')]

compare['crashPctChangeCompareDec'] = compare.Crash_ID / compare.Crash_ID.iloc[0] - 1
compare['fatalPctChangeCompareDec'] = compare.Crash_Fatal_Fl / compare.Crash_Fatal_Fl.iloc[0] - 1

compare['State']='Connecticut'
compare['Country']='United States'

In [36]:
compare

,yearMonth,Crash_ID,Crash_Fatal_Fl,crashPctChangeCompareDec,fatalPctChangeCompareDec,State,Country
11,2019-12,10856,10,0.000000,0.0,Connecticut,United States
12,2020-01,8762,22,-0.192889,1.2,Connecticut,United States
13,2020-02,7593,17,-0.300571,0.7,Connecticut,United States
14,2020-03,5948,13,-0.452100,0.3,Connecticut,United States
15,2020-04,3444,19,-0.682756,0.9,Connecticut,United States
16,2020-05,4178,9,-0.615144,-0.1,Connecticut,United States


In [40]:
with open("CT_PctChangeCompareDec.csv",'w', newline='') as outputfile:
    compare.to_csv(outputfile, header=True, index = False) 